In [1]:
import geopandas as gpd
from sqlalchemy import create_engine

In [2]:
engine = create_engine(   
"mssql+pyodbc:///?odbc_connect="
"Driver%3D%7BODBC+Driver+17+for+SQL+Server%7D%3B"
"Server%3Drliddb.int.lcog.org%2C5433%3B"
"Database%3DRLIDGeo%3B"
"Trusted_Connection%3Dyes%3B"
"ApplicationIntent%3DReadWrite%3B"
"WSID%3Dclwrk4087.int.lcog.org%3B")

In [3]:
sql = '''
SELECT 
stop_number AS number,
stop_name AS name,
longitude,
latitude,
Shape.STAsBinary() AS geom
FROM dbo.LTD_Stop;
'''

In [4]:
transit = gpd.GeoDataFrame.from_postgis(sql, engine, geom_col='geom')

In [5]:
transit.head()

,number,name,longitude,latitude,geom
0,00001,E/S of 58th N of Main,-122.926727,44.046333,POINT (4283261.084 876541.193)
1,00002,E/S of 58th S of D,-122.926224,44.049454,POINT (4283426.923 877674.440)
2,00004,E/S of 58th S of Thurston Rd,-122.926247,44.052246,POINT (4283450.254 878691.941)
3,00006,S/S of Thurston W of 64th,-122.916626,44.052948,POINT (4285984.544 878874.556)
4,00007,S/S of Thurston E of 65th,-122.913231,44.052673,POINT (4286875.484 878748.784)


In [6]:
outpath = r'T:\MPO\RTP\FY20 2045 Update\Data and Resources\PerformanceAnalysis'

In [32]:
transit.crs = "EPSG:2914"

In [7]:
import os

In [33]:
transit.to_file(os.path.join(outpath, "transit_stops.shp"))

In [9]:
# get high frequency stops
path = r'T:\Tableau\tableauTransit\Datasources'

In [10]:
import pandas as pd

In [12]:
on_counts = pd.read_csv(os.path.join(path,'AllPassengerCounts.csv'), low_memory=False)

In [13]:
on_counts.head()

,stop,srv,date,block,trip_end,time,route,dir,stop_name,bus,odometer,ons,offs,load,longitude,latitude,Season,MonthYear
0,00001,wkd,2012-10-05 00:00:00,2,1899-12-31 10:56:00,1899-12-31 10:46:00,11,O,E/S of 58th N of Main,1114,84.26,0,3,5.0,-122.926726,44.046335,Fall 2012,October 2012
1,00001,wkd,2012-10-08 00:00:00,2,1899-12-31 12:03:00,1899-12-31 11:48:00,11,O,E/S of 58th N of Main,336,97.44,0,1,4.0,-122.926726,44.046335,Fall 2012,October 2012
2,00001,wkd,2012-10-31 00:00:00,2,1899-12-31 09:56:00,1899-12-31 09:43:00,11,O,E/S of 58th N of Main,1114,71.05,0,1,4.0,-122.926726,44.046335,Fall 2012,October 2012
3,00001,wkd,2012-10-03 00:00:00,67,1899-12-31 21:36:00,1899-12-31 21:25:00,11,O,E/S of 58th N of Main,1002,137.70,1,0,27.0,-122.926726,44.046335,Fall 2012,October 2012
4,00001,wkd,2012-10-24 00:00:00,60,1899-12-31 11:33:00,1899-12-31 11:20:00,11,O,E/S of 58th N of Main,772,53.77,0,1,9.0,-122.926726,44.046335,Fall 2012,October 2012


In [14]:
list(on_counts)

['stop',
 'srv',
 'date',
 'block',
 'trip_end',
 'time',
 'route',
 'dir',
 'stop_name',
 'bus',
 'odometer',
 'ons',
 'offs',
 'load',
 'longitude',
 'latitude',
 'Season',
 'MonthYear']

In [17]:
out_df = on_counts.groupby('stop_name')[['ons']].sum()

In [22]:
lonlat = on_counts[['stop_name', 'longitude', 'latitude']].drop_duplicates()

In [23]:
lonlat.head()

,stop_name,longitude,latitude
0,E/S of 58th N of Main,-122.926726,44.046335
1214,E/S of 58th S of D,-122.926220,44.049456
2183,E/S of 58th S of Thurston Rd,-122.926243,44.052247
2857,S/S of Thurston W of 64th,-122.916628,44.052949
3588,S/S of Thurston E of 65th,-122.913227,44.052675


In [26]:
df = out_df.merge(lonlat, how='inner', on='stop_name')

In [27]:
df.head()

,stop_name,ons,longitude,latitude
0,"""E"" Street Station",5619,-123.022069,44.050859
1,"""F"" Street Station",5812,-123.023180,44.051158
2,"""Q"" Street Station inbound",11636,-123.026118,44.062726
3,"""Q"" Street Station outbound",6258,-123.026273,44.063299
4,(RRS) E/S River Rd N of River Ave (NE),72133,-123.126966,44.096173


In [35]:
stops_frequency = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [36]:
stops_frequency.crs = "EPSG:2914"

In [37]:
stops_frequency.to_file(os.path.join(outpath, "stops_frequency.shp"))